In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import norm
import datetime
import os
import supertrend_lib as stlib
import sbeta_lib as sbeta
import smacross_lib as smacross
import CAPM_lib as capm
import LR_run_model_lib as LR_run
import LR_model_lib as LR_models
import yfinance as yf
pd.set_option('display.max_rows', 200)
from IPython.display import display, HTML


In [2]:
# symbols_list = eval_df.Stock.values
def get_company_info(symbols_list):
    info_list = list()
    
    for s in symbols_list:
        shandle = yf.Ticker(s)
        info_list.append(shandle.info)

    return info_list
    
# info_list = get_company_info(['aapl','msft'])
# print(info_list)

In [3]:
def add_update(eval_df,values):
    """ Add a a row to dataframe from a list of data points in the same sequence as the columns

    Args:
        eval_df (_type_): DataFrame receiving the new row
        values_list (_type_): A list of values in the same data types and sequance order as the dataframe columns

    Returns:
        _type_: The updated dataframe
    """
    tmp_df = pd.DataFrame([values], columns=eval_df.columns)
    eval_df = pd.concat([eval_df, tmp_df], ignore_index=True)
    return eval_df

In [4]:
def read_stocklist(filename):
    with open(filename) as f:
        stocklist = f.read()
    return stocklist.strip().split(",")


In [5]:
def update_models(stock_list):
    ret = LR_models.update_list_of_models(stock_list=stock_list)
    models_df = pd.DataFrame({'Stock' : stock_list,
                              'Model' : [f"Model{i}" for i in ret]})
    
    # print(models_df)
    return models_df

In [6]:
# stocks_from_file = read_stocklist(filename='./stocks_list.txt')
# stocks_from_file = [s.strip().upper() for s in list(stocks_from_file)]
# dfff = update_models(stocks_from_file)
# dfff

In [7]:
# dfff[dfff['Stock'] == 'AAPL']['Model'].values[0]

In [8]:
def run_supertrend(stock,start_date):
    return stlib.supertrend(stock,start_date=start_date)


In [9]:
def init_eval_table():
    columns = ['Last Run'          ,'Stock'             ,'Last Price'     ,'%Std. Dev.','$Std. Dev.','Supertrend Winner' ,'Supertrend Result','ST Signal_Date', 
               'LR Best_Model'     ,'LR Next_Day Recomm', 'SMA Crossed_Up','SMA_X_Date'        ,'SMA FastXSlow'    , 'Beta'         , 
               '%Sharpe Ratio'                  ,'CAPM'              ,'Daily VaR'  ]

    dtypes = ['datetime64[ns]'       , str                , float            ,str      ,str        , bool              , str               , str            , 
              str                  , str                , str              , 'datetime64[ns]'    , str               , float          , 
              float                , float                , float        ]

    # Initialize the DataFrame with empty rows
    eval_df = pd.DataFrame(columns=columns)

    # Convert the column data types
    for col, dtype in zip(columns, dtypes):
        eval_df[col] = eval_df[col].astype(dtype)
        
    return eval_df

In [10]:
def first_date_N_years_ago(years):
    # Calculate start and end dates N years back
    end_date = datetime.date.today() - datetime.timedelta(days=1)
    start_date = end_date.replace(year=end_date.year - years, month=1, day=1)
    return start_date.strftime("%Y-%m-%d")


In [11]:
def load_sp500_list():
    stocks_list_csv = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp_df = pd.DataFrame()
    
    try:
        wiki_data=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies') # Open the link and download S&P company details in a table
        data = wiki_data[0] # All data is stored in first cell
        sp_df = data.sort_values(by=['Symbol'], ascending=True) # Sort the dataframe on ticker in alphabetical ascending order
    except:
        print("Cannot open file", stocks_list_csv)

    # remove the dotted symbols, they are redundant 
    no_dot_symbols = [i for i in sp_df['Symbol'] if i.find('.')==-1]
    sp_df = sp_df[sp_df['Symbol'].isin(no_dot_symbols)]
    
    # returns count_row, count_col, df
    return sp_df.shape[0], sp_df.shape[1], sp_df


In [12]:
m,n,sp_df = load_sp500_list()
# sp_df[sp_df['Symbol'].isin([i for i in sp_df['Symbol'] if i.find('.')>0])]

In [13]:
def get_sigma(stock_data):
    # Calculate daily price changes
    price_changes = stock_data["Close"].pct_change()

    mu, std = norm.fit(price_changes.dropna())
    return mu, std

In [46]:
def recommendation_table(eval_df,stock_list, lookback_years=1, sma_fast=40, sma_slow=200,run_update_models=False):
    stock_list = [str(s).upper() for s in stock_list]
    # print("Stock List:",stock_list)
    start_date = first_date_N_years_ago(lookback_years)
    today = datetime.datetime.today().strftime("%Y-%m-%d")
    if run_update_models :
        print("Updating Models ...",end='')
        stock_best_model_df = update_models(stock_list=stock_list)
        print("Done!")

    print('Performing Analysis and Recommendations ...',end='')

    for s in stock_list:
        winner,buysell,buysell_date,close_price,stock_data = run_supertrend(s,start_date)
        if(len(stock_data)<2):
            print(f"{s} data not found. Skipping!",end=",")
            continue
        
        print(f"{s}",end=',')
        mean, std_dev = get_sigma(stock_data)
        std_dev_pct = round(std_dev * 100,2)
        std_dev_dlr = round(close_price*std_dev,2)
        sma_sig, sma_date, fastXslow = smacross.sma_xing(stock_data,sma_fast,sma_slow)
        beta, market_data = sbeta.get_beta(stock_data)
        CAPM, VaR, Sharpe = capm.CAPM_VaR(stock_data=stock_data,market_data=market_data,bond_mat_duration = lookback_years,stock_beta=beta)
        Sharpe = round(Sharpe,2)
        LR_recommend = LR_run.get_recommendation(stock=s,lookback=lookback_years)
        LR_recommend_str = f"{LR_recommend[0]},{LR_recommend[1]},{LR_recommend[2]}"
        
        # columns = ['Last_Run'          ,'Stock'             ,'Last_Price'     ,'%Std. Dev.','$Std. Dev.','Supertrend_Winner' ,'Supertrend_Result','ST_Signal_Date', 
        #            'LR_Best_Model'     ,'LR_Next_Day_Recomm', 'SMA_Crossed_Up','SMA_X_Date'             ,'SMA_FastXSlow'     , 'Beta'         , 
        #            'SharpeRatio'                  ,'CAPM'              ,'Daily_VaR'  ]
    
        if run_update_models:
            model_str = stock_best_model_df[stock_best_model_df['Stock'] == s].Model.values[0]
        else:
            model_str = 'N/A'
            
        new_row = [today           , s                  , close_price          ,f"+/-{std_dev_pct}%",f"+/-${std_dev_dlr}",    winner            , buysell         ,    buysell_date ,    
                   model_str       , LR_recommend_str   , sma_sig              , sma_date           , fastXslow          ,    beta              , 
                   Sharpe          , CAPM            , VaR  ]    
        
        eval_df = add_update(eval_df=eval_df,values = new_row)
        
    print("Done!")
    return eval_df

In [15]:
def eval_all_sp500(lookback_years = 1,sma_fast = 40, sma_slow = 200, regenerate_models = False ):
    n,m,sp_df = load_sp500_list()
    stocks_sp500 = sp_df.Symbol.values
    stocks_sp500 = [s.strip().upper() for s in list(stocks_sp500)]

    eval_df = init_eval_table()
    eval_df = recommendation_table(eval_df,
                                stock_list=stocks_sp500, 
                                lookback_years=lookback_years, 
                                sma_fast=sma_fast, 
                                sma_slow=sma_slow,run_update_models=regenerate_models)
    
    return eval_df

In [16]:
def eval_list_from_file(filename='./stocks_list.txt',lookback_years = 1,sma_fast = 40, sma_slow = 200 ,regenerate_models = False):
    stocks_from_file = read_stocklist(filename=filename)
    stocks_from_file = [s.strip().upper() for s in list(stocks_from_file)]
    today = datetime.datetime.today().strftime("%Y-%m-%d")

    eval_df = init_eval_table()
    eval_df = recommendation_table(eval_df,
                                stock_list=stocks_from_file, 
                                lookback_years=lookback_years, 
                                sma_fast=sma_fast, 
                                sma_slow=sma_slow,run_update_models=regenerate_models)
    
    return eval_df

In [31]:
def screen_for_buys(eval_df, ignore_supertrend_winners=False):
        if not ignore_supertrend_winners:
                buys_df = eval_df[ (eval_df['Supertrend Winner']==True) &  
                        (eval_df['Supertrend Result']=='Buy') & 
                        (eval_df['LR Next_Day Recomm'] == 'Buy,Buy,Buy') &
                        (eval_df['SMA Crossed_Up']=='Buy')].sort_values(by=['Supertrend_Winner','Supertrend Result',
                                                                            'ST Signal_Date','SMA Crossed_Up','SMA_X_Date'],
                                                                        ascending=[False,True,False,True,False])
        else:
                buys_df = eval_df[ (eval_df['Supertrend Result']=='Buy') & 
                        (eval_df['LR Next_Day Recomm'] == 'Buy,Buy,Buy') &
                        (eval_df['SMA Crossed_Up']=='Buy')].sort_values(by=['Supertrend Winner','Supertrend Result',
                                                                            'ST Signal_Date','SMA Crossed_Up','SMA_X_Date'],
                                                                        ascending=[False,True,False,True,False])            
        
        return buys_df

In [32]:
def screen_for_sells(eval_df, ignore_supertrend_winners=False):
        if not ignore_supertrend_winners:
                sells_df = eval_df[ (eval_df['Supertrend Winner']==True) &  
                        ((eval_df['Supertrend Result']=='Sell') |
                        (eval_df['LR Next_Day Recomm'] == 'Sell,Sell,Sell') |
                        (eval_df['SMA Crossed_Up']=='Sell')) ].sort_values(by=['Supertrend Winner','Supertrend Result',
                                                                            'ST Signal_Date','SMA Crossed_Up','SMA_X_Date'],
                                                                        ascending=[False,True,False,True,False])
        else:
                sells_df = eval_df[ (eval_df['Supertrend Result']=='Sell') |
                        (eval_df['LR Next_Day Recomm'] == 'Sell,Sell,Sell') |
                        (eval_df['SMA Crossed_Up']=='Sell')].sort_values(by=['Supertrend Winner','Supertrend Result',
                                                                            'ST Signal_Date','SMA Crossed_Up','SMA_X_Date'],
                                                                        ascending=[False,True,False,True,False])            
        
        return sells_df

In [19]:
results_dir = './eval_results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
fname = f'{results_dir}/Eval_Results_{datetime.datetime.today().strftime("%Y_%m_%dT%I%M%S%p")}.csv'

### Evaluate Stocks List

In [48]:
regenerate_models = True
# symbols_file = './stocks_portfolio.txt'
symbols_file = './stocks_list.txt'
# symbols_file = './sectors_etfs.txt'

eval_df = eval_list_from_file(filename=symbols_file,lookback_years=2,regenerate_models=regenerate_models)
# eval_df_from_file = eval_df
# eval_df = eval_all_sp500(lookback_years=2, regenerate_models=regenerate_models)
eval_df.to_csv(fname)

Updating Models ...QQQ,AAPL,MSFT,NVDA,AMD,INTC,META,IBM,QCOM,MU,TSLA,SPY,STX,GOOGL,AMZN,SQ,COIN,MRVL,PLTR,AVGO,DE,CDNS,ADBE,NOW,SMCI,KLAC,TSM,ASML,LRCX,ON,TXN,AMAT,JBL,ADI,MCHP,SWKS,HON,DIA,Done!
Performing Analysis and Recommendations ...QQQ,AAPL,MSFT,NVDA,AMD,INTC,META,IBM,QCOM,MU,TSLA,SPY,STX,GOOGL,AMZN,SQ,COIN,MRVL,PLTR,AVGO,DE,CDNS,ADBE,NOW,SMCI,KLAC,TSM,ASML,LRCX,ON,TXN,AMAT,JBL,ADI,MCHP,SWKS,HON,DIA,Done!


### All Results

In [73]:
display(HTML(eval_df.sort_values(['ST Signal_Date','SMA_X_Date'], ascending=False).to_html(index=False)))

Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-06-11,MU,65.43,+/-2.73%,+/-$1.79,False,Sell,2023-06-08,Model2,"Buy,Buy,Buy",Buy,2023-02-27,"(40, 200)",1.5148,-0.28,9.26,-6.50
2023-06-11,DE,378.87,+/-2.03%,+/-$7.69,True,Buy,2023-06-07,Model1,"Sell,Sell,Sell",Sell,2023-05-03,"(40, 200)",0.8339,2.75,7.16,-4.90
2023-06-11,SQ,64.94,+/-4.38%,+/-$2.84,False,Buy,2023-06-02,Model2,"Sell,Sell,Sell",Sell,2023-04-27,"(40, 200)",2.5378,-2.96,12.42,-10.34
2023-06-11,DIA,339.27,+/-1.0%,+/-$3.41,False,Buy,2023-06-02,Model2,"Sell,Sell,Sell",Buy,2022-12-06,"(40, 200)",0.8003,0.53,7.06,-2.39
2023-06-11,COIN,53.28,+/-5.95%,+/-$3.17,False,Buy,2023-06-01,Model1,"Sell,Sell,Sell",Sell,2023-06-08,"(40, 200)",2.7353,-2.91,-3.75,-14.27
2023-06-11,INTC,31.34,+/-2.27%,+/-$0.71,False,Buy,2023-05-31,Model2,"Buy,Buy,Buy",Buy,2023-05-01,"(40, 200)",1.2324,-3.11,8.39,-5.34
2023-06-11,QCOM,119.11,+/-2.51%,+/-$2.99,False,Buy,2023-05-30,Model2,"Sell,Buy,Buy",Sell,2022-04-12,"(40, 200)",1.5270,-1.07,9.30,-5.92
2023-06-11,TXN,170.59,+/-1.8%,+/-$3.07,False,Buy,2023-05-26,Model2,"Buy,Buy,Buy",Sell,2023-06-06,"(40, 200)",1.1662,0.30,8.19,-4.27
2023-06-11,ADI,182.46,+/-1.99%,+/-$3.63,False,Sell,2023-05-24,Model2,"Sell,Sell,Sell",Buy,2022-12-14,"(40, 200)",1.2855,1.79,8.55,-4.76
2023-06-11,MCHP,80.07,+/-2.57%,+/-$2.06,False,Buy,2023-05-22,Model2,"Buy,Buy,Buy",Buy,2022-12-05,"(40, 200)",1.6627,1.44,9.72,-6.16


In [50]:
if symbols_file == './stocks_list.txt':
    eval_df.to_csv("./stocks_eval_df_table.csv")
elif symbols_file == './sectors_etfs.txt':
    eval_df.to_csv("./sectors_etfs_eval_df_table.csv")
    
print_it = False

def send_df_to_printer(df):
    pdf_filename = "full_eval_df_table.pdf"

    fig, ax =plt.subplots(figsize=(12,4))
    ax.axis('tight')
    ax.axis('off')
    the_table = ax.table(cellText=df.values,colLabels=df.columns,loc='top')

    pp = PdfPages(pdf_filename)
    pp.savefig(fig, bbox_inches='tight')
    pp.close()

    os.system("lpr "+pdf_filename)
    
if (print_it):
    send_df_to_printer(eval_df)

### Buy, Buy, and more Buys

In [74]:
buys_eval_df= screen_for_buys(eval_df=eval_df,ignore_supertrend_winners=True)
print(f"{len(buys_eval_df)} Stocks:")
display(HTML(buys_eval_df.to_html(index=False)))

11 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-06-11,CDNS,229.30,+/-2.28%,+/-$5.23,True,Buy,2023-05-18,Model2,"Buy,Buy,Buy",Buy,2022-12-19,"(40, 200)",1.3836,4.10,8.86,-5.51
2023-06-11,JBL,94.60,+/-2.02%,+/-$1.91,True,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2022-09-14,"(40, 200)",1.2501,6.51,8.45,-4.92
2023-06-11,AMD,124.92,+/-3.33%,+/-$4.16,True,Buy,2023-05-08,Model2,"Buy,Buy,Buy",Buy,2023-03-08,"(40, 200)",1.9640,2.59,10.65,-8.04
2023-06-11,META,264.95,+/-3.17%,+/-$8.41,True,Buy,2022-11-10,Model2,"Buy,Buy,Buy",Buy,2023-02-23,"(40, 200)",1.6811,0.95,9.77,-7.79
2023-06-11,INTC,31.34,+/-2.27%,+/-$0.71,False,Buy,2023-05-31,Model2,"Buy,Buy,Buy",Buy,2023-05-01,"(40, 200)",1.2324,-3.11,8.39,-5.34
2023-06-11,MCHP,80.07,+/-2.57%,+/-$2.06,False,Buy,2023-05-22,Model2,"Buy,Buy,Buy",Buy,2022-12-05,"(40, 200)",1.6627,1.44,9.72,-6.16
2023-06-11,NOW,534.03,+/-2.84%,+/-$15.19,False,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2023-02-24,"(40, 200)",1.6388,0.82,9.64,-6.80
2023-06-11,ON,89.02,+/-3.31%,+/-$2.95,False,Buy,2023-05-01,Model1,"Buy,Buy,Buy",Buy,2022-08-19,"(40, 200)",1.9850,5.98,10.71,-8.09
2023-06-11,QQQ,354.50,+/-1.59%,+/-$5.63,False,Buy,2023-03-21,Model2,"Buy,Buy,Buy",Buy,2023-03-01,"(40, 200)",1.2694,0.99,8.50,-3.78
2023-06-11,GOOGL,122.23,+/-2.06%,+/-$2.52,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-04-25,"(40, 200)",1.3451,2.86,8.74,-4.95


### Save the Buys

In [76]:
eval_df.to_csv("./stocks_buys_eval_df_table.csv")

### Time to Sell!

In [77]:
sells_eval_df= screen_for_sells(eval_df=eval_df,ignore_supertrend_winners=True)
print(f"{len(sells_eval_df)} Stocks:")
display(HTML(sells_eval_df.to_html(index=False)))

26 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-06-11,DE,378.87,+/-2.03%,+/-$7.69,True,Buy,2023-06-07,Model1,"Sell,Sell,Sell",Sell,2023-05-03,"(40, 200)",0.8339,2.75,7.16,-4.90
2023-06-11,ADBE,454.00,+/-2.38%,+/-$10.79,True,Buy,2023-05-19,Model2,"Sell,Sell,Sell",Buy,2023-03-16,"(40, 200)",1.4671,-0.01,9.11,-5.72
2023-06-11,ASML,715.86,+/-2.73%,+/-$19.56,True,Buy,2023-05-17,Model1,"Sell,Sell,Sell",Buy,2022-12-13,"(40, 200)",1.8009,2.79,10.14,-6.57
2023-06-11,SMCI,261.66,+/-3.77%,+/-$9.87,True,Buy,2023-05-03,Model2,"Sell,Sell,Sell",None,None,"(40, 200)",1.3740,10.57,8.83,-9.10
2023-06-11,AAPL,180.96,+/-1.86%,+/-$3.36,True,Buy,2023-01-23,Model2,"Sell,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2928,2.81,8.58,-4.45
2023-06-11,NVDA,387.70,+/-3.53%,+/-$13.7,True,Buy,2023-01-12,Model2,"Sell,Sell,Sell",Buy,2023-01-24,"(40, 200)",2.1791,6.16,11.31,-8.60
2023-06-11,AVGO,804.62,+/-2.07%,+/-$16.69,True,Buy,2022-11-08,Model2,"Sell,Sell,Sell",Buy,2023-01-03,"(40, 200)",1.3404,5.15,8.72,-5.00
2023-06-11,DIA,339.27,+/-1.0%,+/-$3.41,False,Buy,2023-06-02,Model2,"Sell,Sell,Sell",Buy,2022-12-06,"(40, 200)",0.8003,0.53,7.06,-2.39
2023-06-11,SQ,64.94,+/-4.38%,+/-$2.84,False,Buy,2023-06-02,Model2,"Sell,Sell,Sell",Sell,2023-04-27,"(40, 200)",2.5378,-2.96,12.42,-10.34
2023-06-11,COIN,53.28,+/-5.95%,+/-$3.17,False,Buy,2023-06-01,Model1,"Sell,Sell,Sell",Sell,2023-06-08,"(40, 200)",2.7353,-2.91,-3.75,-14.27


### All Roads Lead to UP & Safe

In [78]:
buys_safe = buys_eval_df[(buys_eval_df['Beta']>1) & (buys_eval_df['Beta']<2) ].sort_values('Daily VaR',ascending=False)
print(f"{len(buys_safe)} Stocks:")
display(HTML(buys_safe.to_html(index=False)))

11 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-06-11,QQQ,354.50,+/-1.59%,+/-$5.63,False,Buy,2023-03-21,Model2,"Buy,Buy,Buy",Buy,2023-03-01,"(40, 200)",1.2694,0.99,8.50,-3.78
2023-06-11,MSFT,326.79,+/-1.83%,+/-$5.99,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-03-13,"(40, 200)",1.2525,3.56,8.45,-4.41
2023-06-11,JBL,94.60,+/-2.02%,+/-$1.91,True,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2022-09-14,"(40, 200)",1.2501,6.51,8.45,-4.92
2023-06-11,GOOGL,122.23,+/-2.06%,+/-$2.52,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-04-25,"(40, 200)",1.3451,2.86,8.74,-4.95
2023-06-11,INTC,31.34,+/-2.27%,+/-$0.71,False,Buy,2023-05-31,Model2,"Buy,Buy,Buy",Buy,2023-05-01,"(40, 200)",1.2324,-3.11,8.39,-5.34
2023-06-11,CDNS,229.30,+/-2.28%,+/-$5.23,True,Buy,2023-05-18,Model2,"Buy,Buy,Buy",Buy,2022-12-19,"(40, 200)",1.3836,4.10,8.86,-5.51
2023-06-11,MCHP,80.07,+/-2.57%,+/-$2.06,False,Buy,2023-05-22,Model2,"Buy,Buy,Buy",Buy,2022-12-05,"(40, 200)",1.6627,1.44,9.72,-6.16
2023-06-11,NOW,534.03,+/-2.84%,+/-$15.19,False,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2023-02-24,"(40, 200)",1.6388,0.82,9.64,-6.80
2023-06-11,META,264.95,+/-3.17%,+/-$8.41,True,Buy,2022-11-10,Model2,"Buy,Buy,Buy",Buy,2023-02-23,"(40, 200)",1.6811,0.95,9.77,-7.79
2023-06-11,AMD,124.92,+/-3.33%,+/-$4.16,True,Buy,2023-05-08,Model2,"Buy,Buy,Buy",Buy,2023-03-08,"(40, 200)",1.9640,2.59,10.65,-8.04


### Save the Safe-Buys as Top Picks

In [55]:
buys_safe.to_csv("./top_picks.csv")

### UP the Next Day

In [79]:
LR_Next_Day_Recomm_only = eval_df[eval_df['LR Next_Day Recomm']=='Buy,Buy,Buy'].sort_values('Daily VaR',ascending=False)
print(f"{len(LR_Next_Day_Recomm_only)} Stocks:")
display(HTML(LR_Next_Day_Recomm_only.to_html(index=False)))

15 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-06-11,IBM,135.30,+/-1.42%,+/-$1.93,False,Buy,2023-05-19,Model2,"Buy,Buy,Buy",Sell,2023-03-09,"(40, 200)",0.5409,0.79,6.26,-3.40
2023-06-11,QQQ,354.50,+/-1.59%,+/-$5.63,False,Buy,2023-03-21,Model2,"Buy,Buy,Buy",Buy,2023-03-01,"(40, 200)",1.2694,0.99,8.50,-3.78
2023-06-11,TXN,170.59,+/-1.8%,+/-$3.07,False,Buy,2023-05-26,Model2,"Buy,Buy,Buy",Sell,2023-06-06,"(40, 200)",1.1662,0.30,8.19,-4.27
2023-06-11,MSFT,326.79,+/-1.83%,+/-$5.99,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-03-13,"(40, 200)",1.2525,3.56,8.45,-4.41
2023-06-11,JBL,94.60,+/-2.02%,+/-$1.91,True,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2022-09-14,"(40, 200)",1.2501,6.51,8.45,-4.92
2023-06-11,GOOGL,122.23,+/-2.06%,+/-$2.52,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-04-25,"(40, 200)",1.3451,2.86,8.74,-4.95
2023-06-11,INTC,31.34,+/-2.27%,+/-$0.71,False,Buy,2023-05-31,Model2,"Buy,Buy,Buy",Buy,2023-05-01,"(40, 200)",1.2324,-3.11,8.39,-5.34
2023-06-11,CDNS,229.30,+/-2.28%,+/-$5.23,True,Buy,2023-05-18,Model2,"Buy,Buy,Buy",Buy,2022-12-19,"(40, 200)",1.3836,4.10,8.86,-5.51
2023-06-11,MCHP,80.07,+/-2.57%,+/-$2.06,False,Buy,2023-05-22,Model2,"Buy,Buy,Buy",Buy,2022-12-05,"(40, 200)",1.6627,1.44,9.72,-6.16
2023-06-11,MU,65.43,+/-2.73%,+/-$1.79,False,Sell,2023-06-08,Model2,"Buy,Buy,Buy",Buy,2023-02-27,"(40, 200)",1.5148,-0.28,9.26,-6.50


### Down the Next Day

In [80]:
LR_Next_Day_Sell_only = eval_df[(eval_df['LR Next_Day Recomm']=='Sell,Buy,Buy') | 
                                (eval_df['LR Next_Day Recomm']=='Sell,Sell,Sell') | 
                                (eval_df['LR Next_Day Recomm']=='Sell,Buy,Sell') | 
                                (eval_df['LR Next_Day Recomm']=='Buy,Sell,Sell')]

print(f"{len(LR_Next_Day_Sell_only)} Stocks:")
display(HTML(LR_Next_Day_Sell_only.to_html(index=False)))

23 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-06-11,AAPL,180.96,+/-1.86%,+/-$3.36,True,Buy,2023-01-23,Model2,"Sell,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2928,2.81,8.58,-4.45
2023-06-11,NVDA,387.70,+/-3.53%,+/-$13.7,True,Buy,2023-01-12,Model2,"Sell,Sell,Sell",Buy,2023-01-24,"(40, 200)",2.1791,6.16,11.31,-8.60
2023-06-11,QCOM,119.11,+/-2.51%,+/-$2.99,False,Buy,2023-05-30,Model2,"Sell,Buy,Buy",Sell,2022-04-12,"(40, 200)",1.5270,-1.07,9.30,-5.92
2023-06-11,SPY,429.90,+/-1.18%,+/-$5.08,False,Buy,2023-03-31,Model2,"Sell,Sell,Sell",Buy,2023-02-07,"(40, 200)",1.0024,1.11,7.68,-2.82
2023-06-11,STX,60.89,+/-2.55%,+/-$1.55,False,Buy,2023-05-15,Model2,"Sell,Sell,Sell",Sell,2023-06-07,"(40, 200)",1.2186,0.65,8.35,-6.08
2023-06-11,AMZN,123.43,+/-2.45%,+/-$3.02,False,Buy,2023-03-30,Model1,"Sell,Sell,Sell",Buy,2023-05-23,"(40, 200)",1.5102,-1.27,9.25,-5.79
2023-06-11,SQ,64.94,+/-4.38%,+/-$2.84,False,Buy,2023-06-02,Model2,"Sell,Sell,Sell",Sell,2023-04-27,"(40, 200)",2.5378,-2.96,12.42,-10.34
2023-06-11,COIN,53.28,+/-5.95%,+/-$3.17,False,Buy,2023-06-01,Model1,"Sell,Sell,Sell",Sell,2023-06-08,"(40, 200)",2.7353,-2.91,-3.75,-14.27
2023-06-11,MRVL,60.01,+/-3.64%,+/-$2.18,False,Buy,2023-05-17,Model2,"Sell,Sell,Sell",Buy,2023-06-01,"(40, 200)",2.1553,2.32,11.24,-8.62
2023-06-11,PLTR,15.02,+/-4.43%,+/-$0.67,False,Buy,2023-05-09,Model1,"Sell,Sell,Sell",Buy,2023-03-23,"(40, 200)",1.8063,-0.03,10.16,-10.55


### Supertrend Winners and Still Supertrending

In [81]:
up_Supertrend = eval_df[(eval_df['Supertrend Result'] == 'Buy') & (eval_df['Supertrend Winner'] == True)].sort_values('ST Signal_Date', ascending=False)
print(f"{len(up_Supertrend)} Stocks:")
display(HTML(up_Supertrend.to_html(index=False)))

11 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-06-11,DE,378.87,+/-2.03%,+/-$7.69,True,Buy,2023-06-07,Model1,"Sell,Sell,Sell",Sell,2023-05-03,"(40, 200)",0.8339,2.75,7.16,-4.90
2023-06-11,ADBE,454.00,+/-2.38%,+/-$10.79,True,Buy,2023-05-19,Model2,"Sell,Sell,Sell",Buy,2023-03-16,"(40, 200)",1.4671,-0.01,9.11,-5.72
2023-06-11,CDNS,229.30,+/-2.28%,+/-$5.23,True,Buy,2023-05-18,Model2,"Buy,Buy,Buy",Buy,2022-12-19,"(40, 200)",1.3836,4.10,8.86,-5.51
2023-06-11,ASML,715.86,+/-2.73%,+/-$19.56,True,Buy,2023-05-17,Model1,"Sell,Sell,Sell",Buy,2022-12-13,"(40, 200)",1.8009,2.79,10.14,-6.57
2023-06-11,JBL,94.60,+/-2.02%,+/-$1.91,True,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2022-09-14,"(40, 200)",1.2501,6.51,8.45,-4.92
2023-06-11,AMD,124.92,+/-3.33%,+/-$4.16,True,Buy,2023-05-08,Model2,"Buy,Buy,Buy",Buy,2023-03-08,"(40, 200)",1.9640,2.59,10.65,-8.04
2023-06-11,SMCI,261.66,+/-3.77%,+/-$9.87,True,Buy,2023-05-03,Model2,"Sell,Sell,Sell",None,None,"(40, 200)",1.3740,10.57,8.83,-9.10
2023-06-11,AAPL,180.96,+/-1.86%,+/-$3.36,True,Buy,2023-01-23,Model2,"Sell,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2928,2.81,8.58,-4.45
2023-06-11,NVDA,387.70,+/-3.53%,+/-$13.7,True,Buy,2023-01-12,Model2,"Sell,Sell,Sell",Buy,2023-01-24,"(40, 200)",2.1791,6.16,11.31,-8.60
2023-06-11,META,264.95,+/-3.17%,+/-$8.41,True,Buy,2022-11-10,Model2,"Buy,Buy,Buy",Buy,2023-02-23,"(40, 200)",1.6811,0.95,9.77,-7.79


### Supertrending (Winners  or Not)

In [82]:
up_Supertrend = eval_df[eval_df['Supertrend Result'] == 'Buy'].sort_values('ST Signal_Date', ascending=False)
print(f"{len(up_Supertrend)} Stocks:")
display(HTML(up_Supertrend.to_html(index=False)))

35 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-06-11,DE,378.87,+/-2.03%,+/-$7.69,True,Buy,2023-06-07,Model1,"Sell,Sell,Sell",Sell,2023-05-03,"(40, 200)",0.8339,2.75,7.16,-4.90
2023-06-11,SQ,64.94,+/-4.38%,+/-$2.84,False,Buy,2023-06-02,Model2,"Sell,Sell,Sell",Sell,2023-04-27,"(40, 200)",2.5378,-2.96,12.42,-10.34
2023-06-11,DIA,339.27,+/-1.0%,+/-$3.41,False,Buy,2023-06-02,Model2,"Sell,Sell,Sell",Buy,2022-12-06,"(40, 200)",0.8003,0.53,7.06,-2.39
2023-06-11,COIN,53.28,+/-5.95%,+/-$3.17,False,Buy,2023-06-01,Model1,"Sell,Sell,Sell",Sell,2023-06-08,"(40, 200)",2.7353,-2.91,-3.75,-14.27
2023-06-11,INTC,31.34,+/-2.27%,+/-$0.71,False,Buy,2023-05-31,Model2,"Buy,Buy,Buy",Buy,2023-05-01,"(40, 200)",1.2324,-3.11,8.39,-5.34
2023-06-11,QCOM,119.11,+/-2.51%,+/-$2.99,False,Buy,2023-05-30,Model2,"Sell,Buy,Buy",Sell,2022-04-12,"(40, 200)",1.5270,-1.07,9.30,-5.92
2023-06-11,TXN,170.59,+/-1.8%,+/-$3.07,False,Buy,2023-05-26,Model2,"Buy,Buy,Buy",Sell,2023-06-06,"(40, 200)",1.1662,0.30,8.19,-4.27
2023-06-11,TSLA,244.40,+/-3.85%,+/-$9.4,False,Buy,2023-05-22,Model2,"Buy,Buy,Buy",Sell,2022-10-03,"(40, 200)",1.8480,1.45,10.29,-9.27
2023-06-11,MCHP,80.07,+/-2.57%,+/-$2.06,False,Buy,2023-05-22,Model2,"Buy,Buy,Buy",Buy,2022-12-05,"(40, 200)",1.6627,1.44,9.72,-6.16
2023-06-11,IBM,135.30,+/-1.42%,+/-$1.93,False,Buy,2023-05-19,Model2,"Buy,Buy,Buy",Sell,2023-03-09,"(40, 200)",0.5409,0.79,6.26,-3.40


### Fast SMA Crossed Slow SMA

In [83]:
Crossed_up = eval_df[eval_df['SMA Crossed_Up'] == 'Buy'].sort_values('SMA_X_Date', ascending=False)
print(f"{len(Crossed_up)} Stocks")
display(HTML(Crossed_up.to_html(index=False)))

28 Stocks


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-06-11,MRVL,60.01,+/-3.64%,+/-$2.18,False,Buy,2023-05-17,Model2,"Sell,Sell,Sell",Buy,2023-06-01,"(40, 200)",2.1553,2.32,11.24,-8.62
2023-06-11,AMZN,123.43,+/-2.45%,+/-$3.02,False,Buy,2023-03-30,Model1,"Sell,Sell,Sell",Buy,2023-05-23,"(40, 200)",1.5102,-1.27,9.25,-5.79
2023-06-11,INTC,31.34,+/-2.27%,+/-$0.71,False,Buy,2023-05-31,Model2,"Buy,Buy,Buy",Buy,2023-05-01,"(40, 200)",1.2324,-3.11,8.39,-5.34
2023-06-11,GOOGL,122.23,+/-2.06%,+/-$2.52,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-04-25,"(40, 200)",1.3451,2.86,8.74,-4.95
2023-06-11,PLTR,15.02,+/-4.43%,+/-$0.67,False,Buy,2023-05-09,Model1,"Sell,Sell,Sell",Buy,2023-03-23,"(40, 200)",1.8063,-0.03,10.16,-10.55
2023-06-11,ADBE,454.00,+/-2.38%,+/-$10.79,True,Buy,2023-05-19,Model2,"Sell,Sell,Sell",Buy,2023-03-16,"(40, 200)",1.4671,-0.01,9.11,-5.72
2023-06-11,MSFT,326.79,+/-1.83%,+/-$5.99,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-03-13,"(40, 200)",1.2525,3.56,8.45,-4.41
2023-06-11,AAPL,180.96,+/-1.86%,+/-$3.36,True,Buy,2023-01-23,Model2,"Sell,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2928,2.81,8.58,-4.45
2023-06-11,AMD,124.92,+/-3.33%,+/-$4.16,True,Buy,2023-05-08,Model2,"Buy,Buy,Buy",Buy,2023-03-08,"(40, 200)",1.9640,2.59,10.65,-8.04
2023-06-11,QQQ,354.50,+/-1.59%,+/-$5.63,False,Buy,2023-03-21,Model2,"Buy,Buy,Buy",Buy,2023-03-01,"(40, 200)",1.2694,0.99,8.50,-3.78


### SMA Crossed and in Supertrend

In [71]:
Crossed_up = eval_df[(eval_df['SMA Crossed_Up'] == 'Buy') & (eval_df['Supertrend Result'] == 'Buy') ].sort_values('SMA_X_Date', ascending=False)
print(f"{len(Crossed_up)} Stocks:")
# display(Crossed_up.hide_index())
display(HTML(Crossed_up.to_html(index=False)))

25 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-06-11,MRVL,60.01,+/-3.64%,+/-$2.18,False,Buy,2023-05-17,Model2,"Sell,Sell,Sell",Buy,2023-06-01,"(40, 200)",2.1553,2.32,11.24,-8.62
2023-06-11,AMZN,123.43,+/-2.45%,+/-$3.02,False,Buy,2023-03-30,Model1,"Sell,Sell,Sell",Buy,2023-05-23,"(40, 200)",1.5102,-1.27,9.25,-5.79
2023-06-11,INTC,31.34,+/-2.27%,+/-$0.71,False,Buy,2023-05-31,Model2,"Buy,Buy,Buy",Buy,2023-05-01,"(40, 200)",1.2324,-3.11,8.39,-5.34
2023-06-11,GOOGL,122.23,+/-2.06%,+/-$2.52,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-04-25,"(40, 200)",1.3451,2.86,8.74,-4.95
2023-06-11,PLTR,15.02,+/-4.43%,+/-$0.67,False,Buy,2023-05-09,Model1,"Sell,Sell,Sell",Buy,2023-03-23,"(40, 200)",1.8063,-0.03,10.16,-10.55
2023-06-11,ADBE,454.00,+/-2.38%,+/-$10.79,True,Buy,2023-05-19,Model2,"Sell,Sell,Sell",Buy,2023-03-16,"(40, 200)",1.4671,-0.01,9.11,-5.72
2023-06-11,MSFT,326.79,+/-1.83%,+/-$5.99,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-03-13,"(40, 200)",1.2525,3.56,8.45,-4.41
2023-06-11,AAPL,180.96,+/-1.86%,+/-$3.36,True,Buy,2023-01-23,Model2,"Sell,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2928,2.81,8.58,-4.45
2023-06-11,AMD,124.92,+/-3.33%,+/-$4.16,True,Buy,2023-05-08,Model2,"Buy,Buy,Buy",Buy,2023-03-08,"(40, 200)",1.9640,2.59,10.65,-8.04
2023-06-11,QQQ,354.50,+/-1.59%,+/-$5.63,False,Buy,2023-03-21,Model2,"Buy,Buy,Buy",Buy,2023-03-01,"(40, 200)",1.2694,0.99,8.50,-3.78


### Test Single Stock

In [84]:
stock_to_test = 'swks'
single_stock_df = init_eval_table()
single_stock_list = [stock_to_test]
single_stock_list = ['jbl','aapl']
single_stock_df = recommendation_table(single_stock_df,single_stock_list, lookback_years=5, sma_fast=40, sma_slow=200,run_update_models=True)
display(HTML(single_stock_df.to_html(index=False)))

Updating Models ...JBL,AAPL,Done!
Performing Analysis and Recommendations ...JBL,AAPL,Done!


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-06-11,JBL,94.60,+/-2.36%,+/-$2.23,True,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2022-09-14,"(40, 200)",1.2746,4.42,13.82,-5.74
2023-06-11,AAPL,180.96,+/-2.06%,+/-$3.73,True,Buy,2023-01-23,Model2,"Sell,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2256,5.39,13.44,-5.01


In [63]:
import requests
import textwrap
nasdaq_api_key = 'yYCEL8BqzxYUsgG67FTb'

def get_company_info(ticker_symbol):
    ticker_symbol = str(ticker_symbol).upper()
    # Set the endpoint URL and the API key
    url = "https://api.nasdaq.com/api/company/{}/company-profile"
    api_key = nasdaq_api_key

    # Format the endpoint URL with the ticker symbol
    endpoint_url = url.format(ticker_symbol)

    # Set the headers and parameters for the request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299",
        "Accept-Language": "en-US,en;q=0.9"
    }

    params = {
        "apikey": nasdaq_api_key
    }

    # Send the request and get the response
    response = requests.get(endpoint_url, headers=headers, params=params)

    # Parse the response JSON
    response_json = response.json()

    # Get the company full name and sector
    dic = {
        "Ticker" : ticker_symbol,
        "Full_Name" : response_json["data"]["CompanyName"]['value'],
        "Sector" : response_json["data"]["Sector"]["value"],
        "Industry" : response_json["data"]["Industry"]["value"],
        "Region" : response_json["data"]["Region"]["value"],
        "Address" : response_json["data"]["Address"]["value"],
        "Description" : response_json["data"]["CompanyDescription"]["value"]
        }

    df = pd.DataFrame.from_dict(dic, orient='index').T
    return df

company_info = get_company_info('CHTR')
print(f"Company  : {company_info.loc[0]['Full_Name']}")
print(f"Sector   : {company_info.loc[0]['Sector']}")
print(f"Industry : {company_info.loc[0]['Industry']}")
print(f"Region   : {company_info.loc[0]['Region']}")

wrapped_text = textwrap.wrap(company_info.loc[0]['Description'],100)
for line in wrapped_text:
    print(line)


Company  : Charter Communications, Inc.
Sector   : Telecommunications
Industry : Cable & Other Pay Television Services
Region   : North America
Charter is the product of the 2016 merger of three cable companies, each with a decades-long history
in the business: Legacy Charter, Time Warner Cable, and Bright House Networks. The firm now holds
networks capable of providing television, internet access, and phone services to roughly 54 million
U.S. homes and businesses, around 40% of the country. Across this footprint, Charter serves 29
million residential and 2 million commercial customer accounts under the Spectrum brand, making it
the second-largest U.S. cable company behind Comcast. The firm also owns, in whole or in part,
sports and news networks, including Spectrum SportsNet (long-term local rights to Los Angeles Lakers
games), SportsNet LA (Los Angeles Dodgers), SportsNet New York (New York Mets), and Spectrum News
NY1.


In [64]:
import os
import json

def build_sp500_companies_database():
    n, m, sp_df = load_sp500_list()
    stocks_sp500 = sp_df.Symbol.values
    stocks_sp500 = [s.strip().upper() for s in list(stocks_sp500)]
    
    if not os.path.exists("./company_info"):
        os.makedirs("./company_info")
    
    for s in stocks_sp500:
        company_info = get_company_info(s)
        filename = f"./company_info/{s}.json"
        
        with open(filename, "w") as outfile:
            json.dump(company_info, outfile, indent=4)
        
        print(f"Saved {filename}")
    
    print("Done!")

# build_sp500_companies_database()

In [65]:
sp_df.tail(50)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
446,TXN,Texas Instruments,Information Technology,Semiconductors,"Dallas, Texas",NaN,97476,1930
447,TXT,Textron,Industrials,Aerospace & Defense,"Providence, Rhode Island",1978-12-31,217346,1923
456,TYL,Tyler Technologies,Information Technology,Application Software,"Plano, Texas",2020-06-22,860731,1966
462,UAL,United Airlines Holdings,Industrials,Passenger Airlines,"Chicago, Illinois",2015-09-03,100517,1967
459,UDR,"UDR, Inc.",Real Estate,Residential REITs,"Highlands Ranch, Colorado",2016-03-07,74208,1972
466,UHS,Universal Health Services,Health Care,Health Care Facilities,"King of Prussia, Pennsylvania",2014-09-20,352915,1979
460,ULTA,Ulta Beauty,Consumer Discretionary,Specialty Stores,"Bolingbrook, Illinois",2016-04-18,1403568,1990
465,UNH,UnitedHealth Group,Health Care,Managed Health Care,"Minnetonka, Minnesota",1994-07-01,731766,1977
461,UNP,Union Pacific Corporation,Industrials,Rail Transportation,"Omaha, Nebraska",1957-03-04,100885,1862
463,UPS,United Parcel Service,Industrials,Air Freight & Logistics,"Sandy Springs, Georgia",2002-07-22,1090727,1907
